In [0]:
%run ./00_setup

In [0]:
entities = ["airports", "flights", "customers", "bookings"]

for entity in entities:

    print(f"Starting ingestion for: {entity}")
    df = spark.readStream.format("cloudFiles") \
        .option("cloudFiles.format", "csv") \
        .option("cloudFiles.schemaLocation", f"{bronze_path}/{entity}/checkpoint") \
        .option("cloudFiles.schemaEvolutionMode", "rescue") \
        .load(f"{raw_path}/{entity}")

    query = df.writeStream \
        .format("delta") \
        .outputMode("append") \
        .trigger(once=True) \
        .option("checkpointLocation", f"{bronze_path}/{entity}/checkpoint") \
        .option("path", f"{bronze_path}/{entity}/data") \
        .start()

    query.awaitTermination()
    
    print(f"Completed ingestion for: {entity}")